Before we start, we need to make sure that we have a Kafka cluster running and a topic that produces some streaming data. For simplicity, we will use a single-node Kafka cluster and a topic named `users`. Open the `5.0 user-gen-kafka.ipynb` notebook and execute the cell. This notebook produces a user record every few seconds and put it on a Kafka topic called `users`. 

In [17]:
from delta import  # Delta Lake 라이브러리 임포트 configure_spark_with_delta_pip, DeltaTable
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 col, from_json
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 StructType, StructField, IntegerType, StringType

In [18]:
builder = (SparkSession.builder  # SparkSession 빌더 패턴 시작
           .appName("idempotent-stream-write-delta")  # 애플리케이션 이름 설정
           .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
           .config("spark.executor.memory", "512m")  # Spark 설정 옵션
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Spark 설정 옵션
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Spark 설정 옵션)

spark = configure_spark_with_delta_pip(builder,['org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1']).getOrCreate()  # SparkSession 생성 또는 기존 세션 반환
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

In [19]:
get_ipython().run_line_magic('load_ext', 'sparksql_magic')
get_ipython().run_line_magic('config', 'SparkSql.limit=20')

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic


In [20]:
%%sparksql
CREATE OR REPLACE TABLE default.users (
    id INT,
    name STRING,
    age INT,
    gender STRING,
    country STRING 
) USING DELTA  # Delta Lake 테이블 생성 LOCATION '/opt/workspace/data/delta_lake/idempotent-stream-write-delta/users';

In [21]:
df = (spark.readStream  # 스트리밍 데이터 읽기
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe", "users")
      .option("startingOffsets", "earliest")
      .load(  # 파일 로드))

In [22]:
schema = StructType  # 구조체 타입([
    StructField  # 구조체 필드('id', IntegerType(), True),
    StructField  # 구조체 필드('name', StringType(), True),
    StructField  # 구조체 필드('age', IntegerType(), True),
    StructField  # 구조체 필드('gender', StringType(), True),
    StructField  # 구조체 필드('country', StringType(), True)])

df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정'value', from_json(col('value').cast("STRING"), schema))

In [23]:
df = df.select(  # 컬럼 선택
    col(  # 컬럼 참조'value.id').alias('id'),
    col(  # 컬럼 참조'value.name').alias('name'),
    col(  # 컬럼 참조'value.age').alias('age'),
    col(  # 컬럼 참조'value.gender').alias('gender'),
    col(  # 컬럼 참조'value.country').alias('country'))

In [24]:
query = (df.writeStream  # 스트리밍 데이터 쓰기
   .format("delta")  # Delta Lake 형식으로 저장
   .outputMode(  # 스트리밍 출력 모드 설정"append")
   .option("checkpointLocation", "/opt/workspace/data/delta_lake/idempotent-stream-write-delta/users/_checkpoints/")
   .start("/opt/workspace/data/delta_lake/idempotent-stream-write-delta/users"))

In [25]:
# Define a function writing to two destinations
app_id = 'idempotent-stream-write-delta'
def writeToDeltaLakeTableIdempotent(batch_df, batch_id):
    # location 1
    (batch_df.filter(  # 데이터 필터링"country IN ('India','China')")
     .write
     .format("delta")  # Delta Lake 형식으로 저장
     .mode("append")  # 기존 데이터에 추가
     .option("txnVersion", batch_id)
     .option("txnAppId", app_id)
     .save("/opt/workspace/data/delta_lake/idempotent-stream-write-delta/user_asia"))
    # location 2
    (batch_df.filter(  # 데이터 필터링"country IN ('USA','Canada','Brazil')")
     .write
     .format("delta")  # Delta Lake 형식으로 저장
     .mode("append")  # 기존 데이터에 추가
     .option("txnVersion", batch_id)
     .option("txnAppId", app_id)
     .save("/opt/workspace/data/delta_lake/idempotent-stream-write-delta/user_americas"))

In [26]:
# 적용 the function against the micro-batches using ‘foreachBatch’
write_query = (df
 .writeStream  # 스트리밍 데이터 쓰기
 .format("delta")  # Delta Lake 형식으로 저장
 .queryName("Users By Region")
 .foreachBatch(writeToDeltaLakeTableIdempotent)
 .start()  # 스트리밍 시작)

In [27]:
%%sparksql
SELECT COUNT(*) FROM delta.`/opt/workspace/data/delta_lake/idempotent-stream-write-delta/user_asia`;

count(1)
44


In [28]:
%%sparksql
SELECT COUNT(*) FROM delta.`/opt/workspace/data/delta_lake/idempotent-stream-write-delta/user_americas`;

count(1)
65


In [37]:
%%sparksql
SELECT COUNT(*) FROM delta.`/opt/workspace/data/delta_lake/idempotent-stream-write-delta/user_asia`;

count(1)
45


In [38]:
%%sparksql
SELECT COUNT(*) FROM delta.`/opt/workspace/data/delta_lake/idempotent-stream-write-delta/user_americas`;

count(1)
68


In [39]:
query.stop()
write_query.stop()

In [40]:
spark.stop()  # Spark 세션 종료 - 리소스 정리 